In [4]:
import QuantLib as ql 
import matplotlib.pyplot as plt
%matplotlib inline
ql.__version__

'1.19'

In [54]:
def American_Option(S,K,sigma,r,tau,option_type,q):
    maturity_date = ql.Date(15, 1, 2016)
    calculation_date = maturity_date - tau
    spot_price = S
    strike_price = K
    volatility = sigma # the historical vols or implied vols
    dividend_rate =  q
    risk_free_rate = r
    if option_type == 'Call':
        option_type = ql.Option.Call
    else:
        option_type = ql.Option.Put
        
    day_count = ql.Actual365Fixed()
    calendar = ql.UnitedStates()

    ql.Settings.instance().evaluationDate = calculation_date

    payoff = ql.PlainVanillaPayoff(option_type, strike_price)
    settlement = calculation_date

    am_exercise = ql.AmericanExercise(settlement, maturity_date)
    american_option = ql.VanillaOption(payoff, am_exercise)



    spot_handle = ql.QuoteHandle(
        ql.SimpleQuote(spot_price)
    )
    flat_ts = ql.YieldTermStructureHandle(
        ql.FlatForward(calculation_date, risk_free_rate, day_count)
    )
    dividend_yield = ql.YieldTermStructureHandle(
        ql.FlatForward(calculation_date, dividend_rate, day_count)
    )
    flat_vol_ts = ql.BlackVolTermStructureHandle(
        ql.BlackConstantVol(calculation_date, calendar, volatility, day_count)
    )
    bsm_process = ql.BlackScholesMertonProcess(spot_handle, 
                                               dividend_yield, 
                                               flat_ts, 
                                               flat_vol_ts)

    steps = 200
    binomial_engine = ql.BinomialVanillaEngine(bsm_process, "crr", steps)
    american_option.setPricingEngine(binomial_engine)
    return american_option.NPV()

In [56]:
American_Option(116.32,115.,0.1779,0.02,15,'Call',0.1171875)

1.2671894420422325